# Generación del clasificador para explotación 
## Grado en Ingeniería Informática. Universidad de Burgos
**Autor:** Alicia Olivares Gil

In [25]:
%matplotlib inline 
#para dibujar en el propio notebook
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np
import matplotlib.pyplot as plt #matplotlib.pyplot como plot
import pickle as pk
import loadData as ld
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics as mtr
import seaborn as sns
import tsfresh as tf

### Cargar días de la crisis 1 y 2: 

In [26]:
with open('diacrisis1.pdd','rb') as f: 
    diacrisis1 = pk.load(f) 
print(len(diacrisis1))

121979


In [27]:
with open('diacrisis2.pdd','rb') as f: 
    diacrisis2 = pk.load(f) 
print(len(diacrisis2))

123525


### Función de extracción de características por ventana: 

In [28]:
import tsfresh as tf

def rolling_extract_features(X, window, fc_parameters): 
    features = pd.DataFrame()
    for i in range(len(X)-window): 
        X_rolling = X.iloc[i:i+window]
        features_rolling = tf.extract_features(X_rolling, default_fc_parameters=fc_parameters, column_id='id', column_sort='DateTime')
        features = pd.concat([features,features_rolling],axis=0)
    return features

### Extracción de características de los días de la crisis 1 y 2: 

In [29]:
#características a calcular 
fc_parameters = {
    "agg_linear_trend": [{"attr":"intercept", "chunk_len":5, "f_agg":"var"},{"attr":"stderr", "chunk_len":5, "f_agg":"min"}],
    "symmetry_looking": [{"r":0.6}],
    "change_quantiles":[{"ql":0.2,"qh":1.0,"isabs":False,"f_agg":"var"},{"ql":0.4,"qh":0.8,"isabs":False,"f_agg":"var"},{"ql":0.4,"qh":0.6,"isabs":True,"f_agg":"var"},{"ql":0.0,"qh":0.6,"isabs":True,"f_agg":"mean"},{"ql":0.4,"qh":1.0,"isabs":True,"f_agg":"mean"}],
    "last_location_of_minimum": None, 
    "number_peaks": [{"n":1}]
}

In [30]:
X1,y1 = diacrisis1.drop(['target'],axis=1), diacrisis1['target']
X1['id'] = 1
X1 = X1.reset_index(drop=True)
X2,y2 = diacrisis2.drop(['target'],axis=1), diacrisis2['target']
X2['id'] = 1
X2 = X2.reset_index(drop=True)
X1.head()

,DateTime,P1,P2,P3,P4,P5,P6,id
0,2018-11-09 21:02:53,0.0,18.852459,22.131148,22.950820,13.934426,6.557377,1
1,2018-11-09 21:02:53,0.0,19.672131,22.950820,23.770492,13.934426,7.377049,1
2,2018-11-09 21:02:53,0.0,18.852459,22.131148,22.950820,13.934426,6.557377,1
3,2018-11-09 21:02:53,0.0,19.672131,22.950820,22.950820,13.934426,6.557377,1
4,2018-11-09 21:02:53,0.0,18.852459,22.131148,22.950820,13.934426,6.557377,1


In [31]:
features_diacrisis1 = rolling_extract_features(X1,90,fc_parameters)
features_diacrisis2 = rolling_extract_features(X2,90,fc_parameters)


Feature Extraction:   0%|          | 0/6 [00:00<?, ?it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 434.40it/s]


Feature Extraction:   0%|          | 0/6 [00:00<?, ?it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 475.75it/s]


Feature Extraction:   0%|          | 0/6 [00:00<?, ?it/s]


...

Feature Extraction:   0%|          | 0/6 [00:00<?, ?it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 448.46it/s]


Feature Extraction: 100%|██████████| 6/6 [00:00<00:00, 461.06it/s]

1910


variable,"P1__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""stderr""","P1__agg_linear_trend__f_agg_""var""__chunk_len_5__attr_""intercept""","P1__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","P1__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","P1__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.4","P1__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2","P1__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.4",P1__last_location_of_minimum,P1__number_peaks__n_1,P1__symmetry_looking__r_0.6,...,"P6__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""stderr""","P6__agg_linear_trend__f_agg_""var""__chunk_len_5__attr_""intercept""","P6__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","P6__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","P6__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.4","P6__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2","P6__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.4",P6__last_location_of_minimum,P6__number_peaks__n_1,P6__symmetry_looking__r_0.6
id,,,,,,,,,,,,,,,,,,,,,
1,0.053178,3.726911,1.163048,0.231983,0.0,4.647140,0.0,0.577778,13.0,1.0,...,0.008302,0.025146,0.011228,0.149031,0.0,0.122157,0.0,1.000000,8.0,1.0
1,0.053178,3.569750,1.178980,0.242866,0.0,4.655622,0.0,0.566667,14.0,1.0,...,0.008302,0.015087,0.011077,0.141323,0.0,0.115750,0.0,1.000000,7.0,1.0
1,0.064143,3.222425,1.178980,0.238450,0.0,4.655622,0.0,0.555556,14.0,1.0,...,0.008302,-0.006915,0.021858,0.133435,0.0,0.109373,0.0,0.988889,7.0,1.0
1,0.049745,3.662475,1.195355,0.248829,0.0,4.662238,0.0,0.544444,14.0,1.0,...,0.008302,0.008487,0.021858,0.133435,0.0,0.109373,0.0,0.977778,7.0,1.0
1,0.052657,3.995970,1.212191,0.244464,0.0,4.662238,0.0,0.533333,14.0,1.0,...,0.008302,0.010373,0.021858,0.133435,0.0,0.109373,0.0,0.966667,7.0,1.0


### Añadir target a las características calculadas: 

In [ ]:
features_diacrisis1.reset_index(drop=True)
y=y1[90:].reset_index(drop=True)
features_diacrisis1 = pd.concat([features_diacrisis1,y],axis=1)
features_diacrisis2.reset_index(drop=True)
y=y2[90:].reset_index(drop=True)
features_diacrisis2 = pd.concat([features_diacrisis2,y],axis=1)

### Guardar características: 

In [ ]:
#guardar resultados
with open('features_diacrisis1_final.pdd','wb') as f:
    pk.dump(features_diacrisis1,f)
with open('features_diacrisis2_final.pdd','wb') as f: 
    pk.dump(features_diacrisis2,f)